### Huggingface Endpoints

Imports

In [1]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain import LlamaCpp

Load model

In [15]:
llm = LlamaCpp(
    model_path = '/Users/charmainemahachi/Documents/ZIVA/Phi-3-mini-4k-instruct-fp16.gguf',
    max_tokens=500,
    seed=42,
    verbose=False
)

llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h

### Prompt Template

In [42]:
title_template = """ 
<s><|user|>
When provided with a unique document number and title, {input},
create the single full title for the technical document following the format 'unique document number: Title'.
Capitalise the first letter of every word in the title except for articles i.e. and, the, a, an, is, found in the middle of the title
Return the full title only.
<|end|>
<|assistant|>
"""

In [47]:
summary_prompt_template = """
<s><|user|> Summarise the conversation and update with new lines

current summary: {summary}

new lines of conversation: {new_lines}

New summary:<|end|>
<|assistant|>
"""

summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [43]:
title_prompt = PromptTemplate(
    template=title_template,
    input_variables=['input']
)

### Chain with multiple prompts

In [44]:
from langchain import LLMChain

In [45]:
title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key='title')

In [46]:
title_chain.invoke({"input": " 123 How to make a car"},)

{'input': ' 123 How to make a car', 'title': ' 123: How To Make A Car'}

### Memory

In [48]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

/var/folders/h0/ngy4kv9j5b12994x9km0kzdw0000gn/T/ipykernel_23230/1479079883.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [49]:
chain = LLMChain(
    prompt=title_prompt,
    llm=llm,
    memory=memory
)

In [50]:
chain.invoke({"input": "123 How to make a car"})

{'input': '123 How to make a car',
 'chat_history': '',
 'text': ' 123: How to Make a Car'}

In [52]:
chain.invoke({"input": 'my name is charmaine. The title is 1111 the car is great'})

{'input': 'my name is charmaine. The title is 1111 the car is great',
 'chat_history': ' The conversation revolves around the topic of making a car. The AI acknowledges this query by repeating it back to the user with an emphasis on capitalization for clarity.\n\nNew summary: User inquired about instructions for creating a car, which was echoed and formatted by the AI.',
 'text': ' 1111: The Car Is Great\nanswer: 1111: The Car Is Great'}

In [54]:
chain.invoke({"input": 'What is my name?'})

{'input': 'What is my name?',
 'chat_history': ' Charmaine self-identifies as the author of "1111," acknowledging it for its impressive portrayal of a superior vehicle, while also providing guidance on titling conventions with an example involving quantum computing applications.\n\nNew lines of conversation:\n\nHuman: What is my name?\nAI: Based on your document reference "TX-3021," I will proceed to update the summary accordingly. Here\'s how you might introduce yourself while discussing a new title:\n\n"I am associated with TX-3021, and within this context, let me share my name as well as provide an example of appropriately capitalizing a sophisticated title related to quantum technology."',
 'text': ' Assuming you have provided the unique document number as "TECH-12345" and the title as "recommendations for effective data management practices", here\'s how the full title would be formatted following your instructions:\n\nUnique Document Number: Recommendations For Effective Data Man